## **SỬ DỤNG RAPID API THAY CHO API GPT**

In [17]:
from langchain.llms.base import LLM
from langchain.chat_models.base import BaseChatModel
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import AIMessage, ChatResult, ChatGeneration, BaseMessage
from langchain.callbacks.manager import CallbackManager, CallbackManagerForLLMRun
from langchain.memory import ConversationSummaryMemory
from typing import Optional, List
from dataclasses import dataclass, field
import requests

In [13]:
class RapidAPIChatGPT(BaseChatModel):
    # ✅ Không truyền vào constructor – LangChain tự xử lý
    @property
    def _llm_type(self) -> str:
        return "rapidapi-chat-model"

    def _map_role(self, role: str) -> str:
        if role == "human":
            return "user"
        elif role == "ai":
            return "assistant"
        elif role == "system":
            return "system"
        else:
            return role

    def _generate(self, messages: List[BaseMessage], **kwargs) -> AIMessage:
        print(messages)
        # Convert LangChain messages to API format
        formatted_messages = [{"role": self._map_role(m.type), "content": m.content} for m in messages]
        print(formatted_messages)

        headers = {
            "x-rapidapi-key": "d35f23c2d5msh4c23959a265167cp135bdcjsnc3d3333c3b97",
            "x-rapidapi-host": "chatgpt-42.p.rapidapi.com",
            "Content-Type": "application/json"
        }

        payload = {
            "messages": formatted_messages,
            "web_access": False,
        }

        response = requests.post("https://chatgpt-42.p.rapidapi.com/gpt4", json=payload, headers=headers)
        if response.status_code != 200:
            raise Exception(f"API call failed: {response.status_code} - {response.text}")

        result = response.json()['result']
        ai_message = AIMessage(content=result)
    
        return ChatResult(
            generations=[ChatGeneration(message=ai_message)]
        )   

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain

llm = RapidAPIChatGPT()

prompt_template = ChatPromptTemplate.from_messages(
    [('user', '''Tôi cần mua một thiết bị phù hợp với nhu cầu: {need}''')]
)

chain = prompt_template | llm    # tạo ra một "RunnableSequence", là cách LangChain khuyến khích dùng từ v0.1.0 trở đi.
response = chain.invoke({
    'need': "chơi game và chụp ảnh đẹp, giá dưới 10 triệu"
})


[HumanMessage(content='Tôi cần mua một thiết bị phù hợp với nhu cầu: chơi game và chụp ảnh đẹp, giá dưới 10 triệu', additional_kwargs={}, response_metadata={})]
[{'role': 'user', 'content': 'Tôi cần mua một thiết bị phù hợp với nhu cầu: chơi game và chụp ảnh đẹp, giá dưới 10 triệu'}]


In [16]:
response.content

'Để tìm một thiết bị phù hợp với nhu cầu chơi game và chụp ảnh đẹp trong mức giá dưới 10 triệu đồng, bạn có thể cân nhắc các lựa chọn sau:\n\n### 1. **Điện thoại thông minh**\n- **Xiaomi Redmi Note 11 Pro**:\n  - Màn hình: 6.67 inch FHD+\n  - Chip xử lý: MediaTek Dimensity 920\n  - Camera: Camera chính 108MP, camera góc rộng 8MP, camera macro 2MP, camera xóa phông 2MP\n  - Pin: 5000 mAh, hỗ trợ sạc nhanh\n  - Giá: Khoảng 7-8 triệu đồng\n\n- **Samsung Galaxy A32**:\n  - Màn hình: 6.4 inch FHD+\n  - Chip xử lý: MediaTek Helio G80\n  - Camera: Camera chính 64MP, camera góc rộng 8MP, camera macro 5MP, camera xóa phông 5MP\n  - Pin: 5000 mAh\n  - Giá: Khoảng 6-7 triệu đồng\n\n### 2. **Laptop**\nNếu bạn muốn tìm một chiếc laptop có khả năng chơi game và tương đối tốt cho việc chụp ảnh (thông qua webcam tốt), bạn có thể tham khảo:\n\n- **Asus VivoBook 15 (M514)**:\n  - CPU: AMD Ryzen 5\n  - RAM: 8GB\n  - Card đồ họa: AMD Radeon Graphics\n  - Màn hình: 15.6 inch FHD\n  - Giá: Khoảng 9-10 triệu

## **SỬ DỤNG DEEPSEEK LOCAL**